In [60]:
import random
import os
import numpy as np
import torch
import json
from tqdm import tqdm
import pandas as pd
from transformers import RobertaTokenizer

In [61]:
df = pd.read_csv("/shared/3/projects/benlitterer/localNews/NetworkMVP/translatedCleaned.tsv", sep="\t")

In [62]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [63]:
# truncate text from either head or tail part
def trunc_text(text, trunc_pos, length):

    text_ids = tokenizer.encode(text)[1:-1]

    if trunc_pos == 'head':
        text_trunc_ids = text_ids[:length]
    elif trunc_pos == 'tail':
        text_trunc_ids = text_ids[-length:]

    text_trunc_tokens = tokenizer.decode(text_trunc_ids)
    #text_trunc_back_sent = ''.join([x.replace('_', ' ') for x in text_trunc_tokens])[:-1]

    return text_trunc_tokens

In [64]:
trunc_text(df["text1"][0], "tail", 20)

' Associated Press. All rights reserved. This material may not be published, broadcast, rewritten or redistributed.'

In [67]:
df = df.dropna(subset=["title1", "title2", "text1", "text2"])

In [68]:
df["titleText1"] = df["title1"] + df["text1"]
df["titleText2"] = df["title2"] + df["text2"]

In [69]:
df["text1Head"] = df["titleText1"] .apply(trunc_text, args=["head", 200])
df["text1Tail"] = df["text1"].apply(trunc_text, args=["tail", 56])

df["text2Head"] = df["titleText2"].apply(trunc_text, args=["head", 200])
df["text2Tail"] = df["text2"].apply(trunc_text, args=["tail", 56])

In [73]:
df["text1Merged"] = df["text1Head"] + df["text1Tail"]
df["text2Merged"] = df["text2Head"] + df["text2Tail"]

In [74]:
df.to_csv("/home/blitt/projects/localNews/data/processed/translated_200_56.tsv", sep="\t")